# Supervised Learning: Gradient Boosting Regressor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from supervised_learning.cross_validation import PanelDataSplit
from supervised_learning.cross_validation import search_best_model
from sample_panel.merge_datasets import merge_bank_macro_datasets

from supervised_learning.estimate_errors import estimate_median_relative_error
from supervised_learning.estimate_errors import estimate_errors

## Preparing a Data Sample

### Loading data

In [3]:
# Load bank panel data
bank_data = pd.read_csv('df_response_vars.csv')

In [4]:
# Load macroeconomic data
macro_data = pd.read_csv('macro_features.csv')
macro_columns = macro_data.columns

# Factors with lags are not used in the model. Remove factors with lags
new_macro_columns = [col for col in macro_columns if '_lag' not in col]
macro_data = macro_data[new_macro_columns]

In [5]:
# Load PCA components
pca_data = pd.read_csv('macro_pca_df.csv')

In [6]:
# Load additional macro variables
macro_data1 = pd.read_csv('macro_most_inf_df.csv')
# Clean column names
macro_data1.columns = [col.replace('\n', ' ') for col in macro_data1.columns]

### Merging bank panel and macroeconomic time series

In [7]:
# Merge the bank panel and macroeconomic indicators
result_df = merge_bank_macro_datasets(bank_data, macro_data, pca_data, macro_data1)

In [8]:
# Delete Nans values due to the lag of the response variable
result_df.dropna(subset=['Provision_Lag1'], inplace=True)
result_df.reset_index(drop=True, inplace=True)

In [9]:
result_df.head(5)

,Report Date,IDRSSD,Financial Institution Name,Provision for Loan Lease Losses as % of Aver. Assets,Real GDP growth,Nominal GDP growth,Unemployment rate,Unemployment rate change,3-month Treasury rate change,BBB corporate yield,...,Nominal disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Households_Net_Worth,Developing Asia real GDP growth,Euro area inflation,Developing Asia inflation,Provision_Lag1
0,2003-03-31,12311,"HUNTINGTON NATIONAL BANK, THE",0.54,2.1,4.1,5.9,0.2,-0.5,6.2,...,2.9,4.2,1.2,2.9,4.2,45531292.0,6.6,3.3,3.6,0.74
1,2003-03-31,14409,CITIZENS BANK OF MASSACHUSETTS,0.32,2.1,4.1,5.9,0.2,-0.5,6.2,...,2.9,4.2,1.2,2.9,4.2,45531292.0,6.6,3.3,3.6,0.30
2,2003-03-31,17147,"FIRST MERCHANTS BANK, NATIONAL ASSOCIATION",1.77,2.1,4.1,5.9,0.2,-0.5,6.2,...,2.9,4.2,1.2,2.9,4.2,45531292.0,6.6,3.3,3.6,0.31
3,2003-03-31,23504,"BRIDGEHAMPTON NATIONAL BANK, THE",0.00,2.1,4.1,5.9,0.2,-0.5,6.2,...,2.9,4.2,1.2,2.9,4.2,45531292.0,6.6,3.3,3.6,0.05
4,2003-03-31,30810,DISCOVER BANK,5.93,2.1,4.1,5.9,0.2,-0.5,6.2,...,2.9,4.2,1.2,2.9,4.2,45531292.0,6.6,3.3,3.6,6.20


### Additional features

In [11]:
# Adding Fixed Effects
# Assuming 'Financial Institution Name' is a categorical variable, so we can one-hot encode it
result_df = pd.get_dummies(result_df, columns=['IDRSSD'], drop_first=True)

## Supervised model

In [13]:
# Response variable column
y_col = 'Provision for Loan Lease Losses as % of Aver. Assets'

In [14]:
scaler = StandardScaler()
gb_model = GradientBoostingRegressor(random_state=42)

pipeline = Pipeline(steps=[("scaler", scaler), ("gb_model", gb_model)])

### Train-test split

In [15]:
# The last year is for test,remaining data - for train
data_set_train = result_df[result_df['Report Date']<='2021-12-31'].copy()
data_set_test = result_df[result_df['Report Date']>'2021-12-31'].copy()

#### Removing outliers from the train set

In [16]:
lower_limit = np.percentile(data_set_train[y_col], 0.5)
upper_limit = np.percentile(data_set_train[y_col], 99)

data_set_train = data_set_train[(data_set_train[y_col]<=upper_limit)&(data_set_train[y_col]>=lower_limit)].copy()
data_set_train.reset_index(drop=True, inplace=True)

## Optimizing Model Selection: Factor Selection and Hyperparameter Tuning
The analysis on time series data for the "average" bank indicates that the response variable is most influenced by its lag of one quarter. Moreover, the residuals of the autoregressive time series model AR(1) exhibit the strongest correlation with the following factors: 'Real GDP growth_ema3', 'BBB corporate yield', '3-month Treasury rate change', 'Dow Jones Total Stock Market Index change', 'Market Volatility Index', 'Market Volatility Index change'. 

'Nominal GDP growth_ema1.75' also displays a high correlation with the AR(1) residuals, but it was excluded from the models based on economic reasoning. The decision to exclude it based on the fact that high Nominal GDP values do not always mean a healthy economy; during periods of high inflation, high Nominal GDP values can reflect inflation rather than economic well-being.

To select the optimal model, we created a list of model variants that include these influential factors (models1-models11). Addidtionally, we added model12 and model13 based on the results of usupervised learning.

Model selection and hyperparameter tuning are performed using cross-validation, allowing us to determine the best model and optimal hyperparameter values.

In [18]:
# Custom cross-validation split for panel data
panel_cv = PanelDataSplit(test_size=4, date_axis=data_set_train['Report Date'], n_splits=5)

In [27]:
# Models
models = {'model1': ['Provision_Lag1', 'Real GDP growth_ema3', 'BBB corporate yield'],
          'model2': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index'],
          'model3': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change'],
          'model4': ['Provision_Lag1', 'Real GDP growth_ema3', 'Dow Jones Total Stock Market Index change'],
          'model5': ['Provision_Lag1', 'Real GDP growth_ema3', '3-month Treasury rate change'],
          'model6': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index', 'BBB corporate yield'],
          'model7': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index', '3-month Treasury rate change'],
          'model8': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change', 'BBB corporate yield'],
          'model9': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change', '3-month Treasury rate change'],
          'model10': ['Provision_Lag1', 'Real GDP growth_ema3', 'Market Volatility Index change', '3-month Treasury rate change', 'BBB corporate yield'],
          'model11': ['Provision_Lag1', 'Real GDP growth_ema3', 'BBB corporate yield', '3-month Treasury rate change'],
          'model12': ['Provision_Lag1'] + list(pca_data.columns[:-1]), #PCA components
          'model13': ['Provision_Lag1', 'Japan bilateral dollar exchange rate (yen/USD)', 
                      'Euro area bilateral dollar exchange rate (USD/euro)',
                      'NBER_Recession_Indicator_Peak_through_Trough', 'Commercial_Banks_Treasury_and_Agency_Securities',
                      'Real disposable income growth', 'U.K. bilateral dollar exchange rate (USD/pound)', 
                      'Unemployment rate', 'BBB corporate yield', 'Households_Net_Worth', 'Euro area inflation', 
                      'Market Volatility Index', 'Developing Asia inflation'],
                             
         }

In [28]:
# Grid to search over to be able get better test results and reduce overfitting

# Define the parameters
param_grid = {
    'gb_model__n_estimators': [100, 200, 300, 400, 500],
    'gb_model__learning_rate': [0.05, 0.1, 0.2, 0.3],
    'gb_model__max_depth': [3, 4, 5]
}


# Initialize the grid search
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='r2', cv=panel_cv)

In [29]:
best_model_name, best_score, best_model, models_results, estimators = \
    search_best_model(data_set_train, models, grid_search, 
                      y_col, [col for col in result_df.columns if col.startswith('IDRSSD_')])

### Results of model selection

In [30]:
best_model_name

'model4'

In [31]:
# Factors of the best model
models[best_model_name]

['Provision_Lag1',
 'Real GDP growth_ema3',
 'Dow Jones Total Stock Market Index change']

In [32]:
# Pipeline for the best model
best_model

Pipeline(steps=[('scaler', StandardScaler()),
                ('gb_model', GradientBoostingRegressor(random_state=42))])

In [36]:
models_results['Cross-Validation R^2 Standard Error of the Mean'] = \
    models_results['Cross-Validation R^2 std'] / panel_cv.get_n_splits()**0.5

models_results

,Cross-Validation R^2,Cross-Validation R^2 std,Best Hyperparameters,Cross-Validation R^2 Standard Error of the Mean
model1,0.641872,0.441640,"{'gb_model__learning_rate': 0.05, 'gb_model__m...",0.197507
model2,0.652643,0.419762,"{'gb_model__learning_rate': 0.05, 'gb_model__m...",0.187723
model3,0.657573,0.417645,"{'gb_model__learning_rate': 0.1, 'gb_model__ma...",0.186777
model4,0.679486,0.387952,"{'gb_model__learning_rate': 0.1, 'gb_model__ma...",0.173497
model5,0.637102,0.430668,"{'gb_model__learning_rate': 0.1, 'gb_model__ma...",0.192601
model6,0.649688,0.429142,"{'gb_model__learning_rate': 0.1, 'gb_model__ma...",0.191918
model7,0.643928,0.418659,"{'gb_model__learning_rate': 0.1, 'gb_model__ma...",0.187230
model8,0.651683,0.425119,"{'gb_model__learning_rate': 0.1, 'gb_model__ma...",0.190119
model9,0.635401,0.439048,"{'gb_model__learning_rate': 0.05, 'gb_model__m...",0.196348
model10,0.650965,0.400298,"{'gb_model__learning_rate': 0.2, 'gb_model__ma...",0.179019


In [42]:
models_results.to_csv('GB_models_results.csv')

## Final model after hyperparameter tuning: Test sample performance

In [37]:
# Best model features
features_all = models[best_model_name] + [col for col in result_df.columns if col.startswith('IDRSSD_')]

X_train = data_set_train[features_all]
X_test = data_set_test[features_all]

In [43]:
# Fit the best model using the whole train set
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_train)

### Train sample performance

In [44]:
estimate_errors(y_train, y_pred, lower_limit, upper_limit)

,measure
R squared,0.882779
RMSE,0.297373
"median relative error, %",17.976709


### Test sample performance

In [45]:
y_pred = best_model.predict(X_test)

In [46]:
# All errors together (excluding outliers)
estimate_errors(y_test, y_pred, lower_limit, upper_limit)

,measure
R squared,0.821973
RMSE,0.221627
"median relative error, %",40.290357
